In [1]:
import os
import time
import math

import numpy as np
import pandas as pd

import tqdm

import warnings
warnings.filterwarnings("ignore")

import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AdamW, 
    get_linear_schedule_with_warmup,
    DataCollatorWithPadding,
    Trainer, 
    TrainingArguments,
    AutoModelForCausalLM
)

from peft import (
    LoraConfig, 
    get_peft_model, 
    TaskType,
    PeftModel
)

import datasets 

In [2]:
# Define a function that can print the trainable parameters 
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [3]:
full_data = pd.read_csv('hackaton_result_dataset.csv', sep=';')
full_data['text'] = full_data['model_annotation']
full_data = full_data.drop(['model_annotation', 'human_markup', 'audio_path', 'Unnamed: 4'], axis=1)
print(f"We have {len(full_data)} samples") # Number of data we have
full_data.head(20)

We have 6508 samples


,label,text
0,1,давай по россии значит на коленях быстро блять...
1,0,ну разве можно так с телефоном поступает
2,0,у меня нет с собой в полном адресе я щас дома ...
3,0,а я здесь кто я санитар
4,0,дежурный по кузьминскому военнокомату
5,0,не понял
6,1,че и так
7,0,я записываю сигнал
8,1,я записала а вы там наговорили а номер кто
9,0,ааа так так так


In [4]:
from sklearn.model_selection import train_test_split

# Split it when augmented data is ready
X_train, X_val, y_train, y_val = train_test_split(full_data["text"],
                                                  full_data["label"],
                                                  test_size=0.3,
                                                  stratify=full_data["label"],
                                                  random_state=100500)
print(f"We have {len(X_train)} training samples")
print(f"We have {len(X_val)} validation samples")
print("----------------------------")
count = full_data["label"].value_counts()
print(f"Number of Essays written by Human: {count[0]}")
print(f"Number of Essays generated by LLM: {count[1]}")

X_train.reset_index(drop = True, inplace = True)
y_train.reset_index(drop = True, inplace = True)
X_val.reset_index(drop = True, inplace = True)
y_val.reset_index(drop = True, inplace = True)

We have 4555 training samples
We have 1953 validation samples
----------------------------
Number of Essays written by Human: 3902
Number of Essays generated by LLM: 2606


In [5]:
MODEL_PATH = 'ai-forever/ruBert-base'
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, return_dict=True, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
lora_config = LoraConfig(
    r=8, # Rank Number
    lora_alpha=32, # Alpha (Scaling Factor)
    lora_dropout=0.05, # Dropout Prob for Lora
    target_modules=["query", "key", "value"], # Which layer to apply LoRA, usually only apply on MultiHead Attention Layer
    bias='none',
    task_type=TaskType.SEQ_CLS # Seqence to Classification Task
)
peft_model = get_peft_model(model, lora_config)

# Reduced trainble parameters
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 443906
all model parameters: 178752772
percentage of trainable model parameters: 0.25%


In [7]:
# Tokenize function
def tokenize_func(data):
    return tokenizer(
            data['texts'],
            max_length=512,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )


In [8]:
train_dataset = datasets.Dataset.from_pandas(pd.DataFrame({"texts":X_train,"labels":y_train}))
train_dataset = train_dataset.map(
    tokenize_func,
    batched=True,
    remove_columns=["texts"]
)
train_dataset

Map:   0%|          | 0/4555 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4555
})

In [9]:
# Tokenize the Validation Data
val_dataset = datasets.Dataset.from_pandas(pd.DataFrame({"texts":X_val,"labels":y_val}))
val_dataset = val_dataset.map(
    tokenize_func,
    batched=True,
    remove_columns=["texts"]
)

val_dataset

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1953
})

In [10]:
# Define Eval Metric
def metrics(eval_prediction):
    logits, labels = eval_prediction
    pred = np.argmax(logits, axis=1)
    auc_score = roc_auc_score(labels, pred)
    return {"Val-AUC": auc_score}

train_batch_size = 32
eval_batch_size = 32

# Define training Args
peft_training_args = TrainingArguments(
    output_dir='./result-distilbert-lora',
    logging_dir='./logs-distilbert-lora',
#     auto_find_batch_size=True,
    learning_rate=1e-4,
    per_device_train_batch_size=train_batch_size, # You can adjust this value base on your available GPU, You may encounter "out of memory" error if this value is too lartge
    per_device_eval_batch_size=eval_batch_size, # You can adjust this value base on your available GPU, You may encounter "out of memory" error if this value is too lartge
    num_train_epochs=50,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=10,
    weight_decay=0.01,
    seed=100500,
    fp16=True, # Only use with GPU
    report_to='none'
)   

# Define Optimzer
optimizer = AdamW(peft_model.parameters(), 
                  lr=1e-4,
                  no_deprecation_warning=True)

# Define Scheduler
n_epochs = peft_training_args.num_train_epochs
total_steps = n_epochs * math.ceil(len(train_dataset) / train_batch_size / 2)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=total_steps)

# Data Collator
collator = DataCollatorWithPadding(
    tokenizer=tokenizer, 
    padding="longest"
)

# Define Trainer
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=train_dataset, # Training Data
    eval_dataset=val_dataset, # Evaluation Data
    tokenizer=tokenizer,
    compute_metrics=metrics,
    optimizers=(optimizer,lr_scheduler),
    data_collator=collator
)

print(f"Total Steps: {total_steps}")
# Train the model
peft_trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Total Steps: 3600


Step,Training Loss,Validation Loss,Val-auc
10,0.682700,0.667827,0.502301
20,0.663200,0.659067,0.518700
30,0.662700,0.649475,0.529153
40,0.653300,0.635560,0.555143
50,0.633000,0.616163,0.586239
60,0.614400,0.591092,0.639273
70,0.588300,0.571697,0.666335
80,0.570900,0.562186,0.675932
90,0.538600,0.552133,0.687428
100,0.526900,0.548702,0.689990



KeyboardInterrupt



In [29]:
# Path to save the fine-tuned model
peft_model_path="peft-lora-local-bs32-epoch50"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('peft-lora-local-bs16-epoch5/tokenizer_config.json',
 'peft-lora-local-bs16-epoch5/special_tokens_map.json',
 'peft-lora-local-bs16-epoch5/vocab.txt',
 'peft-lora-local-bs16-epoch5/added_tokens.json',
 'peft-lora-local-bs16-epoch5/tokenizer.json')

In [ ]:
def load_fine_tuned_model(peft_model_path):
    peft_model_base = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, return_dict=True, num_labels=2)
    peft_tokenizer = AutoTokenizer.from_pretrained(peft_model_path)

    peft_model = PeftModel.from_pretrained(peft_model_base, 
                                           peft_model_path,
                                           is_trainable=False)

    return peft_model, peft_tokenizer
